In [1]:
# Try and do the aml exercise independently
# Create/USe a workspace

from azureml.core import Workspace

In [2]:
# Get the Workspace
ws = Workspace.from_config()

In [3]:
# print information abouyt the workspace
ws.name

'quick-starts-ws-152869'

In [5]:
print(f'Name: {ws.name}, Subscription Id: {ws.subscription_id}, Resorce Group: {ws.resource_group}, Location: {ws.location}')

Name: quick-starts-ws-152869, Subscription Id: d7f39349-a66b-446e-aba6-0053c2cf1c11, Resorce Group: aml-quickstarts-152869, Location: southcentralus


In [6]:
# Create an experiment for the run
from azureml.core import Experiment

exp = Experiment(workspace=ws, name="my-automl-exp")

In [7]:
exp

Name,Workspace,Report Page,Docs Page
my-automl-exp,quick-starts-ws-152869,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
# Next, we will create/attach a compute cluster
# Access the cluster. if non-existent, then create one
compute_cluster_name = "my-cc"

from azureml.core.compute import ComputeTarget
from azureml.exceptions import ComputeTargetException
from azureml.core.compute import AmlCompute




In [22]:
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
except ComputeTargetException:
    # Cluster does not exist - create it
    prov_cfg = AmlCompute.provisioning_configuration(vm_size="Standard_DS3_v2", max_nodes=4, description="Pipeline cfg")

    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, prov_cfg)

In [23]:
# Wait for compute cluster to be built
compute_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
# We next to retrieve the Dataset
# 'https://raw.githubusercontent.com/Azure/MachineLearningNotebooks/master/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/bike-no.csv'
# Once retrieved - this dataset will subsequently be used for the model

# Loop through the WorkSpace datasets and see if our datset already exists
# if not, create the dataset


In [ ]:
# We next specify the Pipeline, starting with the configuration
